# PHRASE EXTRACTION FOR REMINDERS

## NLP PART

In [1]:
#import all the necessary packages.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn import svm,neighbors,linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [2]:
data = pd.read_csv('training_data.tsv', header=0,delimiter="\t",quoting=3)
listsent=list(data['sent'])
listlabel=list(data['label'])
l=[]
for i,label in enumerate(listlabel):
    if label=='Not Found':
        pass
    else:
        l.append([listsent[i],label])
 ###### identifing pattern with words on both side ######
l1=[]
for i in l:
        name = i[1]#the label
        name = re.sub('[^A-Za-z0-9]+', ' ', name)#removing any special charecters
        sub = '(\w*)\W*('+name+')\W*(\w*)'#identifing words around name
        str1 = i[0]#the complete sentence
        for i in re.findall(sub, str1, re.I):
            l1.append ([" ".join([x for x in i if x != ""]),name])# some_list is a list of lists with the patter -> <word1><Label><word2>
print(l1)

[['daily wake up call', 'wake up call'], ['for recharge', 'recharge'], ['7pm on 8 Jan', 'on 8 Jan'], ['What is', 'What'], ['to meet sonal miss at', 'meet sonal miss'], ['to purchase shoe polish liquid', 'purchase shoe polish'], ['remember my loves birthday this', 'my loves birthday'], ['dmello meeting with sujit sir remind him Tomorrow', 'meeting with sujit sir remind him'], ['to msg babbu at', 'msg babbu'], ['to pay LIC Premium on', 'pay LIC Premium'], ['to go to bank at', 'go to bank'], ['to buy eggs on', 'buy eggs'], ['for internal audit review meeting at', 'internal audit review meeting'], ['of SBI card payment of rs 4465 by', 'SBI card payment of rs 4465'], ['to go through basic codings and share markets', 'go through basic codings and share markets'], ['to fill a file at', 'fill a file'], ['to work on design qualification documents', 'work on design qualification documents'], ['to send pics to Rajeshwari in', 'send pics to Rajeshwari'], ['for a work', 'a work'], ['to start my stu

In [3]:
###### removing the lable from the string to create a regex bucket
some_pattern_list=[]
for double_list in l1:
    if double_list[1] in double_list[0]:
        some_pattern_list.append(double_list[0].replace(double_list[1], " "))
    else:
        pass
pattern_list1=((Counter(some_pattern_list)).most_common())#converts the Counter dictutionary to list of tuples (pattern,freq)
pattern_list=[tup[0] for tup in pattern_list1]#create a list of patterns
print(pattern_list)

['to  ', 'to   at', 'for  ', 'to   on', 'to   tomorrow', 'about  ', 'to   today', '  at', ' ', 'to   in', 'for   at', 'me  ', 'reminder  ', '  on', 'about   at', 'of  ', 'to   by', 'to   from', 'u  ', 'to   every', 'to   after', 'for   on', '  is', 'of   at', '  will', '  today', 'about   on', 'is  ', '  for', 'that  ', 'you  ', '  reminder', 'to   for', 'a  ', 'of   on', 'know  ', '  in', '  tomorrow', 'me   at', 'for   today', 'about   tomorrow', 'Is  ', 'Please  ', '  after', 'have  ', 'for   tomorrow', 'add  ', 'to   tonight', 'Venue  ', 'for   in', 'to   before', 'on  ', 'need  ', 'am  ', '  every', '  by', 'schedule  ', 'be  ', 'of   tomorrow', 'some  ', '  from', 'tomorrow  ', 'me   tomorrow', 'About  ', 'to   to', 'remind  ', 'm  ', '  please', 'all  ', 'to   this', 'to   tommorow', 'a   at', 'to   everyday', 'me   in', 'for   for', 'about   in', 'To  ', 'Reminder  ', 'to   daily', 'pm  ', 'me   today', 'me   on', 'For  ', 'my  ', 'set  ', 'of   by', 'to   and', 'will  ', 'Pls 

In [4]:
###### identifing patterns with two words on the left ######
some_new_list=[]
for i in l:
    name = i[1]
    name = re.sub('[^A-Za-z0-9]+', ' ', name)
    sub = '(\w*)\W*(\w*)\W*('+name+')'
    str1 = i[0]
    for i in re.findall(sub, str1, re.I):
        some_new_list.append ([" ".join([x for x in i if x != ""]),name])  
some_pattern_list_new=[]
for double_list in some_new_list:
    if double_list[1] in double_list[0]:
        some_pattern_list_new.append(double_list[0].replace(double_list[1], " "))
    else:
        pass
pattern_list2=((Counter(some_pattern_list_new)).most_common())
pattern_list_2=[tup[0] for tup in pattern_list2]
print(pattern_list_2)


['me to  ', ' ', 'me about  ', 'me for  ', 'reminder to  ', 'have to  ', 'reminder for  ', 'me of  ', 'want to  ', 'Remind to  ', 'Reminder to  ', 'Remind me  ', 'pm to  ', 'Reminder for  ', 'need to  ', 'tomorrow to  ', 'Thanks for  ', 'remind me  ', 'remind to  ', 'Please  ', 'am to  ', 'today to  ', 'Is  ', 'pm for  ', 'Can u  ', 'today for  ', 'I have  ', 'tell me  ', 'reminder of  ', 'Remind about  ', 'me that  ', 'I need  ', 'Can you  ', 'Need to  ', 'am for  ', 'Reminder  ', 'to know  ', '30 to  ', 'have a  ', 'me on  ', 'tomorrow for  ', 'pm about  ', 'know about  ', 'I am  ', 'a reminder  ', 'can u  ', 'I m  ', 'remainder to  ', 'to add  ', 'Reminder about  ', 'morning to  ', 'the reminder  ', 'you to  ', 'How to  ', 'reminder is  ', 'About  ', '6pm to  ', 'me abt  ', 'remind for  ', 'I had  ', 'Remind  ', '30pm to  ', 'Pls  ', 'Will u  ', 'Want to  ', 'u to  ', 'everyday to  ', 'For  ', '30am to  ', 'going to  ', 'for to  ', 'me tomorrow  ', '30 for  ', 'Remind of  ', 'Is it 

In [5]:
  len_2_words=int((len(pattern_list_2))*.20)# taking only the top 20% of the all the patterns created
print(len_2_words)

219


In [6]:
data2= pd.read_csv('evaldata.csv', names=['sent','label','nothing1','nothing2'])
data2.drop(['nothing1','nothing2','label'],axis=1)
data2.head(5)

,sent,label,nothing1,nothing2
0,Reminder for daily reporting and attendance on...,NaN,NaN,NaN
1,How will you remind me let me know,NaN,NaN,NaN
2,Can u remind me till 11,NaN,NaN,NaN
3,Call at 9 am every day,NaN,NaN,NaN
4,Remind me to bid for flights on 13th december ...,NaN,NaN,NaN


In [7]:
master_list=[]
sent_list=[]
d= list(data2['sent'])
for text in d:
    found = 0
    small_master_list=[]
    sent_list.append(text)
    for pat in pattern_list:
        boundaries=pat.split()
        if len(boundaries)==2:
            m = re.search(boundaries[0]+' '+'(.+?)'+' '+boundaries[1],str(text))
            if m:
                found=m.group(1)
                small_master_list.append(found)
    for pat in pattern_list_2[:len_2_words]:
        boundaries=pat.split()
        if len(boundaries)==2:
            m =re.search(boundaries[0]+' '+boundaries[1]+'(.*)',str(text))
            if m:
                found=m.group(1)
                small_master_list.append(found)
    if found==0 : # if none of the patterns match give it as Not found
        small_master_list.append('Not Found')
    master_list.append(small_master_list)
print(master_list)

[['daily reporting and', 'daily reporting and attendance', 'daily reporting and attendance on mob app', 'for daily reporting and', 'daily reporting and attendance on mob app daily', 'reporting', 'for daily reporting and attendance', ' daily reporting and attendance on mob app daily 9:00 am'], [' let me know', '', ' me let me know'], [' till 11', ' remind me till 11', ' me till 11'], [' every day'], ['bid for flights on 13th december', 'bid for flights', 'flights on 13th december', 'flights', 'bid', 'to bid for flights on 13th december', 'to bid for flights', '13th december', 'me to bid for flights on 13th december', 'bid for flights on 13th december', 'me to bid for flights', 'flights on', 'bid for flights on', 'to bid for flights on 13th december at', ' bid for flights on 13th december at 6 pm', ' to bid for flights on 13th december at 6 pm'], ['morrorw buy bell for papu', 'morrorw buy bell for papu', 'morrorw buy bell for papu'], ['book b tickets tonight', 'book b tickets', 'book b t

In [8]:
no_no_words=['on','for','to','at','by'] # list of words which if occured in the output will be penalised while giving a score
final_output=[]
############ selecting one from the options
for options in master_list:
    if len(options)==1:
        final_output.append(options[0])#if only one pattern extracted use it 
    else:
        sent_score_list=[] # else score all the options to select the best one
        for option in options:
            l=option.split()
            score = 0
            score = len(l)
            for word in l:
                if word in no_no_words:
                    score = score -3 #penalise the no_no_words
                else:
                    pass
            sent_score_list.append(score)
        m = max(sent_score_list)
        indx=[i for i, j in enumerate(sent_score_list) if j == m] # returns a list of all the index which have max score
        index = indx[-1]#pick the last element as an index 
        final_output.append(options[index])
print(final_output)

[' daily reporting and attendance on mob app daily 9:00 am', ' me let me know', ' remind me till 11', ' every day', '13th december', 'morrorw buy bell for papu', ' book b tickets tonight by 10 pm', ' remind me to masturbate', 'Not Found', ' 30am please. Ignore the 12 20am one', '....2:00 pm ...today for meeting', '', 'vth d', 'see. Concert', 'gain after half an hour', 'payment and phone', ' tomorrow 11 am to pack gift for my college sir.', '1st dec to take appointment of those 2 customer', '0 5', ' morning', ' print the log sheet out today', 'up a recuring', ' ur reminder', 'remind me every Tuesday n Thursday', ' remind me something! Thank you', 'Not Found', 'Not Found', 'pray daily', 'Not Found', ' of talking with RICS TODAY AT 11 AM ', ' complete month', 'Not Found', " take my friend's car for wash on 12th before she comes back", 'me next month 16th for hepatitis B vaccination', 'Not Found', " attend reception of Kishor Bhai's Son on 28th November", 'Not Found', 'Not Found', 'Not Fou

In [9]:
tups=zip(sent_list, final_output)
final_list = [list(l) for l in tups]
print(final_list)

[['Reminder for daily reporting and attendance on mob app daily 9:00 am', ' daily reporting and attendance on mob app daily 9:00 am'], ['How will you remind me let me know', ' me let me know'], ['Can u remind me till 11', ' remind me till 11'], ['Call at 9 am every day', ' every day'], ['Remind me to bid for flights on 13th december at 6 pm', '13th december'], ['Reminder 11am tomorrorw buy bell for papu', 'morrorw buy bell for papu'], ['Remind me to book b tickets tonight by 10 pm', ' book b tickets tonight by 10 pm'], ['Can you remind me to masturbate', ' remind me to masturbate'], ['Reminder on invoice preparation', 'Not Found'], ['1 alarm at 1 30am please. Ignore the 12 20am one', ' 30am please. Ignore the 12 20am one'], ['One more reminder....2:00 pm ...today for meeting', '....2:00 pm ...today for meeting'], ['call sagar @5:30 pm', ''], ["I'm upset vth d reminder service", 'vth d'], ['Remind me to see. Concert at 7pm', 'see. Concert'], ['Remind me again after half an hour', 'gain 

# ML PART

In [10]:
data = pd.read_csv('training_data.tsv',header=0,delimiter="\t" ,quoting=3)

In [11]:
print ('Number of data points : ', data.shape[0], \
       'Number of features/variables:', data.shape[1])

Number of data points :  9819 Number of features/variables: 2


In [12]:
data.columns

Index(['sent', 'label'], dtype='object')

In [13]:
data.head(10)

,sent,label
0,Make remainder,Not Found
1,Set a reminder on date 23rd November'2016,Not Found
2,I need a daily wake up call,wake up call
3,remind me 6 pm today eveng,Not Found
4,Hi Pls to make one reminder for me,Not Found
5,Remind me at 28 December for recharge,recharge
6,Can u help me with reminder..,Not Found
7,Can u pls remind me at 7pm on 8 Jan,on 8 Jan
8,What is my next reminder?,What
9,Can u please set me an alarm at 7.30 am,Not Found


In [14]:
print(data['sent'].describe())

count                              9818
unique                             9811
top       Remind me at 12noon spice jet
freq                                  2
Name: sent, dtype: object


In [15]:
print(data['sent'].unique())

['Make remainder' "Set a reminder on date 23rd November'2016"
 'I need a daily wake up call' ...
 'Saumya remind me of going to attend a meeting on 1st of march 2017 at10am'
 'Can u schedule fr every coming year'
 'Tume acha kaam mila ha koi remainder set ni krta hoga pehle recahrge vala busy tha ab shopping vali shraddha busy ha muje reply ni kr rhi']


In [16]:
print(data['label'].describe())

count          9819
unique         5322
top       Not Found
freq           3912
Name: label, dtype: object


In [17]:
def  notfound(x):
    if x=='Not Found':
        return 0
    return 1
#changing reviews with score less than 3 to be positive and vice-versa
actualdata= data['label']
positiveNegative= actualdata.map(notfound) 
data['label'] = positiveNegative
print("Number of data points in our data", data.shape)
data.head(10)

Number of data points in our data (9819, 2)


,sent,label
0,Make remainder,0
1,Set a reminder on date 23rd November'2016,0
2,I need a daily wake up call,1
3,remind me 6 pm today eveng,0
4,Hi Pls to make one reminder for me,0
5,Remind me at 28 December for recharge,1
6,Can u help me with reminder..,0
7,Can u pls remind me at 7pm on 8 Jan,1
8,What is my next reminder?,1
9,Can u please set me an alarm at 7.30 am,0


In [18]:
data.duplicated(subset={"sent"}).value_counts()

False    9812
True        7
dtype: int64

In [19]:
data=data.drop_duplicates(subset={"sent"},keep="first")

In [20]:
data2=pd.read_csv('evaldata.csv',names=['sent1','label1','hel','hello'])
data2=data2.drop(['label1','hel','hello'],axis=1)
data2.head(5)

,sent1
0,Reminder for daily reporting and attendance on...
1,How will you remind me let me know
2,Can u remind me till 11
3,Call at 9 am every day
4,Remind me to bid for flights on 13th december ...


# BAG OF WORDS

In [25]:
title_vectorizer = CountVectorizer()
title_vectorizer.fit(data['sent'].values.astype('U'))
title_features = title_vectorizer.transform(data['sent'].values.astype('U'))
title_features2=title_vectorizer.transform(data2['sent1'].values.astype('U'))
print(title_features.get_shape())
print(title_features2.get_shape())

(9812, 6364)
(1000, 6364)


In [27]:
Xtrain=title_features
Ytrain=data['label']
Xtest=title_features2
clf =svm.LinearSVC(loss='hinge').fit(Xtrain,Ytrain )
result = clf.predict(Xtest)
output = pd.DataFrame(data={"sent": data2['sent1'], "label": result})
predicted_result=list(result) # converting the n dimentional array into a list
###### output of the classisfier using BOW model ######
output.to_csv(('Bag_of_Words_model_new.csv'), index=False, quoting=3, escapechar='\\')
output.head(5)

,sent,label
0,Reminder for daily reporting and attendance on...,1
1,How will you remind me let me know,0
2,Can u remind me till 11,0
3,Call at 9 am every day,1
4,Remind me to bid for flights on 13th december ...,1


In [28]:
import pickle
f = open('trainedmodels/my_BOWclassifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
f = open('trainedmodels/my_BOWvectorizer.pickle', 'wb')
pickle.dump(title_vectorizer, f)
f.close()

In [29]:
final_result=[]
y=list(result)
for idx,prediction in enumerate(y):
    if (prediction==0):
        final_result.append('Not Found') # if classifier identifies the sentence to be labled as Not Found it has power to over write the extracter
    else:
        final_result.append(final_list[idx][1])#if the classifier identifies it to have a phrase then we use the extracted phrase
print(final_result)

[' daily reporting and attendance on mob app daily 9:00 am', 'Not Found', 'Not Found', ' every day', '13th december', 'morrorw buy bell for papu', ' book b tickets tonight by 10 pm', ' remind me to masturbate', 'Not Found', 'Not Found', '....2:00 pm ...today for meeting', 'Not Found', 'Not Found', 'see. Concert', 'Not Found', 'payment and phone', ' tomorrow 11 am to pack gift for my college sir.', '1st dec to take appointment of those 2 customer', '0 5', 'Not Found', ' print the log sheet out today', 'Not Found', 'Not Found', 'remind me every Tuesday n Thursday', ' remind me something! Thank you', 'Not Found', 'Not Found', 'pray daily', 'Not Found', ' of talking with RICS TODAY AT 11 AM ', 'Not Found', 'Not Found', " take my friend's car for wash on 12th before she comes back", 'me next month 16th for hepatitis B vaccination', 'Not Found', " attend reception of Kishor Bhai's Son on 28th November", 'Not Found', 'Not Found', 'Not Found', ' take print out', ' add reminder', 'renew my insu

In [30]:
output = pd.DataFrame(data={"sent": data2['sent1'], "label": np.asarray(final_result)})
output.to_csv(('output/Bag_of_Words_output.csv'), index=False, quoting=3, escapechar='\\')
output.head(30)

,sent,label
0,Reminder for daily reporting and attendance on...,daily reporting and attendance on mob app dai...
1,How will you remind me let me know,Not Found
2,Can u remind me till 11,Not Found
3,Call at 9 am every day,every day
4,Remind me to bid for flights on 13th december ...,13th december
5,Reminder 11am tomorrorw buy bell for papu,morrorw buy bell for papu
6,Remind me to book b tickets tonight by 10 pm,book b tickets tonight by 10 pm
7,Can you remind me to masturbate,remind me to masturbate
8,Reminder on invoice preparation,Not Found
9,1 alarm at 1 30am please. Ignore the 12 20am one,Not Found


## NGRAM

In [31]:
vectorizer = CountVectorizer(ngram_range=(1,2)) # n-grams Bag of word
vectorizer.fit(data['sent'].values.astype('U'))
train_data_features = vectorizer.transform(data['sent'].values.astype('U')) # expects a list of strings
train_data_features2=vectorizer.transform(data2['sent1'].values.astype('U'))
np.asarray(train_data_features) #Tfidf expects an array hence we convert it
np.asarray(train_data_features2)
###### representing the n-grams wrt to the frequency of occurence ######
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_data_features)
X_test_tfidf=tfidf_transformer.fit_transform(train_data_features2)
print(X_train_tfidf.get_shape())
print(X_test_tfidf.get_shape())

(9812, 32070)
(1000, 32070)


In [32]:
Xtrain=X_train_tfidf
Ytrain=data['label']
Xtest=X_test_tfidf
clf =svm.LinearSVC(loss='hinge').fit(Xtrain,Ytrain )
result = clf.predict(Xtest)
output = pd.DataFrame(data={"sent": data2['sent1'], "label": result})
predicted_result=list(result) # converting the n dimentional array into a list
###### output of the classisfier using BOW model ######
output.to_csv(('NgramBag_of_Words_model_new.csv'), index=False, quoting=3, escapechar='\\')
output.head(5)

,sent,label
0,Reminder for daily reporting and attendance on...,1
1,How will you remind me let me know,1
2,Can u remind me till 11,0
3,Call at 9 am every day,1
4,Remind me to bid for flights on 13th december ...,1


In [33]:
import pickle
f = open('trainedmodels/my_NgramBOWclassifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
f = open('trainedmodels/my_NgramBOWvectorizer.pickle', 'wb')
pickle.dump(TfidfTransformer, f)
f.close()

In [34]:
final_result=[]
y=list(result)
for idx,prediction in enumerate(y):
    if (prediction==0):
        final_result.append('Not Found') # if classifier identifies the sentence to be labled as Not Found it has power to over write the extracter
    else:
        final_result.append(final_list[idx][1])#if the classifier identifies it to have a phrase then we use the extracted phrase
print(final_result)

[' daily reporting and attendance on mob app daily 9:00 am', ' me let me know', 'Not Found', ' every day', '13th december', 'morrorw buy bell for papu', ' book b tickets tonight by 10 pm', ' remind me to masturbate', 'Not Found', 'Not Found', '....2:00 pm ...today for meeting', 'Not Found', 'Not Found', 'see. Concert', 'Not Found', 'payment and phone', ' tomorrow 11 am to pack gift for my college sir.', '1st dec to take appointment of those 2 customer', '0 5', 'Not Found', ' print the log sheet out today', 'Not Found', 'Not Found', 'remind me every Tuesday n Thursday', 'Not Found', 'Not Found', 'Not Found', 'pray daily', 'Not Found', ' of talking with RICS TODAY AT 11 AM ', 'Not Found', 'Not Found', " take my friend's car for wash on 12th before she comes back", 'me next month 16th for hepatitis B vaccination', 'Not Found', " attend reception of Kishor Bhai's Son on 28th November", 'Not Found', 'Not Found', 'Not Found', ' take print out', ' add reminder', 'renew my insurance of car', '

In [35]:
output = pd.DataFrame(data={"sent": data2['sent1'], "label": np.asarray(final_result)})
output.to_csv(('output/NgamBag_of_Words_output.csv'), index=False, quoting=3, escapechar='\\')
output.head(30)

,sent,label
0,Reminder for daily reporting and attendance on...,daily reporting and attendance on mob app dai...
1,How will you remind me let me know,me let me know
2,Can u remind me till 11,Not Found
3,Call at 9 am every day,every day
4,Remind me to bid for flights on 13th december ...,13th december
5,Reminder 11am tomorrorw buy bell for papu,morrorw buy bell for papu
6,Remind me to book b tickets tonight by 10 pm,book b tickets tonight by 10 pm
7,Can you remind me to masturbate,remind me to masturbate
8,Reminder on invoice preparation,Not Found
9,1 alarm at 1 30am please. Ignore the 12 20am one,Not Found


## TFIDF

In [36]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_vectorizer.fit(data['sent'].values.astype('U'))
tfidf_title_features= tfidf_title_vectorizer.transform(data['sent'].values.astype('U'))
tfidf_title_features2=tfidf_title_vectorizer.transform(data2['sent1'].values.astype('U')) 

In [37]:
Xtrain=tfidf_title_features
Ytrain=data['label']
Xtest=tfidf_title_features2
clf =svm.LinearSVC(loss='hinge').fit(Xtrain,Ytrain )
result = clf.predict(Xtest)
output = pd.DataFrame(data={"sent": data2['sent1'], "label": result})
predicted_result=list(result) # converting the n dimentional array into a list
###### output of the classisfier using BOW model ######
output.to_csv(('TFIDF_model_new.csv'), index=False, quoting=3, escapechar='\\')
output.head(5)

,sent,label
0,Reminder for daily reporting and attendance on...,1
1,How will you remind me let me know,0
2,Can u remind me till 11,0
3,Call at 9 am every day,1
4,Remind me to bid for flights on 13th december ...,1


In [38]:
import pickle
f = open('trainedmodels/TFIDFBOWclassifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
f = open('trainedmodels/TFIDFBOWvectorizer.pickle', 'wb')
pickle.dump(TfidfTransformer, f)
f.close()

In [39]:
final_result=[]
y=list(result)
for idx,prediction in enumerate(y):
    if (prediction==0):
        final_result.append('Not Found') # if classifier identifies the sentence to be labled as Not Found it has power to over write the extracter
    else:
        final_result.append(final_list[idx][1])#if the classifier identifies it to have a phrase then we use the extracted phrase
print(final_result)

[' daily reporting and attendance on mob app daily 9:00 am', 'Not Found', 'Not Found', ' every day', '13th december', 'morrorw buy bell for papu', ' book b tickets tonight by 10 pm', ' remind me to masturbate', 'Not Found', 'Not Found', '....2:00 pm ...today for meeting', 'Not Found', 'vth d', 'see. Concert', 'Not Found', 'payment and phone', ' tomorrow 11 am to pack gift for my college sir.', '1st dec to take appointment of those 2 customer', '0 5', 'Not Found', ' print the log sheet out today', 'Not Found', 'Not Found', 'remind me every Tuesday n Thursday', 'Not Found', 'Not Found', 'Not Found', 'pray daily', 'Not Found', ' of talking with RICS TODAY AT 11 AM ', ' complete month', 'Not Found', " take my friend's car for wash on 12th before she comes back", 'me next month 16th for hepatitis B vaccination', 'Not Found', " attend reception of Kishor Bhai's Son on 28th November", 'Not Found', 'Not Found', 'Not Found', ' take print out', ' add reminder', 'renew my insurance of car', 'Not 

In [40]:
output = pd.DataFrame(data={"sent": data2['sent1'], "label": np.asarray(final_result)})
output.to_csv(('output/TFIDF_output.csv'), index=False, quoting=3, escapechar='\\')
output.head(30)

,sent,label
0,Reminder for daily reporting and attendance on...,daily reporting and attendance on mob app dai...
1,How will you remind me let me know,Not Found
2,Can u remind me till 11,Not Found
3,Call at 9 am every day,every day
4,Remind me to bid for flights on 13th december ...,13th december
5,Reminder 11am tomorrorw buy bell for papu,morrorw buy bell for papu
6,Remind me to book b tickets tonight by 10 pm,book b tickets tonight by 10 pm
7,Can you remind me to masturbate,remind me to masturbate
8,Reminder on invoice preparation,Not Found
9,1 alarm at 1 30am please. Ignore the 12 20am one,Not Found


## WORD2VEC

In [42]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.

'''
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
'''

#if you do NOT have RAM >= 12GB, use the code below.
with open('word2vec_model', 'rb') as handle:
    model = pickle.load(handle)

In [43]:
def get_word_vec(sentence):
    vec = []
    for i in sentence.split():
        if i in vocab:
                vec.append(model[i])
        else:
            # if the word in our courpus is not there in the google word2vec corpus, we are just ignoring it
            vec.append(np.zeros(shape=(300,)))
    # we will return a numpy array of shape (#number of words in title * 300 ) 300 = len(w2v_model[word])
    # each row represents the word2vec representation of each word (weighted/avg) in given sentance 
    return  np.array(vec)

In [44]:
vocab = model.keys()
# this function will add the vectors of each word and returns the avg vector of given sentance
def build_avg_vec(sentence, num_features):
    # sentace: its title of the apparel
    # num_features: the lenght of word2vec vector, its values = 300
    featureVec = np.zeros((num_features,), dtype="float32")
    # we will intialize a vector of size 300 with all zeros
    # we add each word2vec(wordi) to this fetureVec
    nwords = 0
    for word in sentence.split():
        nwords += 1
        if word in vocab:
                featureVec = np.add(featureVec, model[word])
    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    # returns the avg vector of given sentance, its of shape (1, 300)
    return featureVec

In [45]:
doc_id = 0
w2v_title = []
# for every title we build a avg vector representation
for i in data['sent']:
    w2v_title.append(build_avg_vec(str(i), 300))
    doc_id += 1

# w2v_title = np.array(# number of doc in courpus * 300), each row corresponds to a doc 
w2v_title = np.array(w2v_title)
w2v_title.shape

(9812, 300)

In [46]:
doc_id = 0
w2v_title2 = []
# for every title we build a avg vector representation
for i in data2['sent1']:
    w2v_title2.append(build_avg_vec(str(i), 300))
    doc_id += 1

# w2v_title = np.array(# number of doc in courpus * 300), each row corresponds to a doc 
w2v_title2 = np.array(w2v_title2)
w2v_title2.shape

(1000, 300)

In [47]:
Xtrain=w2v_title
Ytrain=data['label']
Xtest=w2v_title2
clf =svm.LinearSVC(loss='hinge').fit(Xtrain,Ytrain )
result = clf.predict(Xtest)
output = pd.DataFrame(data={"sent": data2['sent1'], "label": result})
predicted_result=list(result) # converting the n dimentional array into a list
###### output of the classisfier using BOW model ######
output.to_csv(('TFIDF_model_new.csv'), index=False, quoting=3, escapechar='\\')
output.head(5)

,sent,label
0,Reminder for daily reporting and attendance on...,1
1,How will you remind me let me know,1
2,Can u remind me till 11,0
3,Call at 9 am every day,0
4,Remind me to bid for flights on 13th december ...,1


In [48]:
import pickle
f = open('trainedmodels/word2vecclassifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
f = open('trainedmodels/word2vecvectorizer.pickle', 'wb')
pickle.dump(w2v_title, f)
f.close()

In [49]:
final_result=[]
y=list(result)
for idx,prediction in enumerate(y):
    if (prediction==0):
        final_result.append('Not Found') # if classifier identifies the sentence to be labled as Not Found it has power to over write the extracter
    else:
        final_result.append(final_list[idx][1])#if the classifier identifies it to have a phrase then we use the extracted phrase
print(final_result)

[' daily reporting and attendance on mob app daily 9:00 am', ' me let me know', 'Not Found', 'Not Found', '13th december', 'morrorw buy bell for papu', ' book b tickets tonight by 10 pm', ' remind me to masturbate', 'Not Found', 'Not Found', '....2:00 pm ...today for meeting', '', 'vth d', 'Not Found', 'Not Found', 'payment and phone', ' tomorrow 11 am to pack gift for my college sir.', '1st dec to take appointment of those 2 customer', '0 5', 'Not Found', ' print the log sheet out today', 'Not Found', 'Not Found', 'remind me every Tuesday n Thursday', ' remind me something! Thank you', 'Not Found', 'Not Found', 'pray daily', 'Not Found', ' of talking with RICS TODAY AT 11 AM ', ' complete month', 'Not Found', " take my friend's car for wash on 12th before she comes back", 'me next month 16th for hepatitis B vaccination', 'Not Found', 'Not Found', 'Not Found', 'Not Found', 'Not Found', ' take print out', 'Not Found', 'renew my insurance of car', 'Not Found', 'Not Found', 'Not Found', '

In [52]:
output = pd.DataFrame(data={"sent": data2['sent1'], "label": np.asarray(final_result)})
output.to_csv(('output/word2vec_output.csv'), index=False, quoting=3, escapechar='\\')
output.head(30)

,sent,label
0,Reminder for daily reporting and attendance on...,daily reporting and attendance on mob app dai...
1,How will you remind me let me know,me let me know
2,Can u remind me till 11,Not Found
3,Call at 9 am every day,Not Found
4,Remind me to bid for flights on 13th december ...,13th december
5,Reminder 11am tomorrorw buy bell for papu,morrorw buy bell for papu
6,Remind me to book b tickets tonight by 10 pm,book b tickets tonight by 10 pm
7,Can you remind me to masturbate,remind me to masturbate
8,Reminder on invoice preparation,Not Found
9,1 alarm at 1 30am please. Ignore the 12 20am one,Not Found
